In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  gdrive  sample_data


In [7]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/GB/Introduction to Natural Language Processing/lesson_9/1c.txt'

In [8]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 167914 characters


In [9]:
print(text[:500])

Перем СоединениеИсточник, ОрганизацияИсточник, СкладИсточник, КонтрагентПокупатель, ДоговорПокупателя, КонтрагентПокупательПустышек, ДоговорПокупателяПустышек, 
		ПустойСкладИсточник, БазаИсточникНоваяКонфигурация, СтавкиНДСИсточник,ВерсияБазыИсточник;
Перем ДопДанныеИсточник,ДанныеШапкиИсточник;
Перем СоединениеПриемник, ОрганизацияПриемник, СкладПриемник, КонтрагентПоставщик,	ДоговорПоставщика, ПроцентНаценки, ПустойСкладПриемник, БазаПриемникНоваяКонфигурация, СтавкиНДСПриемник;
Перем Дан


In [10]:
text = text + text

In [11]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

140 unique characters


In [13]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [14]:
text_as_int, text, len(text_as_int), len(text)

(array([ 90, 112, 123, ..., 115,   2,   1]),
 'Перем СоединениеИсточник, ОрганизацияИсточник, СкладИсточник, КонтрагентПокупатель, ДоговорПокупателя, КонтрагентПокупательПустышек, ДоговорПокупателяПустышек, \r\n\t\tПустойСкладИсточник, БазаИсточникНоваяКонфигурация, СтавкиНДСИсточник,ВерсияБазыИсточник;\r\nПерем ДопДанныеИсточник,ДанныеШапкиИсточник;\r\nПерем СоединениеПриемник, ОрганизацияПриемник, СкладПриемник, КонтрагентПоставщик,\tДоговорПоставщика, ПроцентНаценки, ПустойСкладПриемник, БазаПриемникНоваяКонфигурация, СтавкиНДСПриемник;\r\nПерем ДанныеШапкиПриемник,ДопДанныеПриемник,ВерсияБазыПриемник;\r\nПерем ПродажиВОтдельнойБазе, СоединениеПродаж, ОрганизацияПродаж, ЭтоСписокБелыхПоставщиков, БазаПродажНоваяКонфигурация, СтавкиНДСБазаПродаж, ВерсияБазыПродаж;\r\nПерем ЛС,ПС, ТаблицаПродано, ТаблицаКуплено, Сообщение, МассивПоставщиков, ИсключаемБезНДСДиски, НеСоздаватьНовые, НастройкиПрогружены;\r\nПерем МаксимизироватьИспользованиеПустых;\r\nПерем ДатаНачалаАнализа,ДатаОкончани

### train and target

In [15]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

П
е
р
е
м


In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Перем СоединениеИсточник, ОрганизацияИсточник, СкладИсточник, КонтрагентПокупатель, ДоговорПокупателя'
', КонтрагентПокупательПустышек, ДоговорПокупателяПустышек, \r\n\t\tПустойСкладИсточник, БазаИсточникНовая'
'Конфигурация, СтавкиНДСИсточник,ВерсияБазыИсточник;\r\nПерем ДопДанныеИсточник,ДанныеШапкиИсточник;\r\nПе'
'рем СоединениеПриемник, ОрганизацияПриемник, СкладПриемник, КонтрагентПоставщик,\tДоговорПоставщика, П'
'роцентНаценки, ПустойСкладПриемник, БазаПриемникНоваяКонфигурация, СтавкиНДСПриемник;\r\nПерем ДанныеШа'


In [17]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [18]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Перем СоединениеИсточник, ОрганизацияИсточник, СкладИсточник, КонтрагентПокупатель, ДоговорПокупател'
Target data: 'ерем СоединениеИсточник, ОрганизацияИсточник, СкладИсточник, КонтрагентПокупатель, ДоговорПокупателя'


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [20]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [21]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [22]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [23]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [60]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [61]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [62]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 140) # (batch_size, sequence_length, vocab_size)


In [63]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 128)         17920     
_________________________________________________________________
gru_8 (GRU)                  (None, None, 1024)        3545088   
_________________________________________________________________
gru_9 (GRU)                  (None, None, 1024)        6297600   
_________________________________________________________________
gru_10 (GRU)                 (None, None, 1024)        6297600   
_________________________________________________________________
dense_4 (Dense)              (None, None, 140)         143500    
Total params: 16,301,708
Trainable params: 16,301,708
Non-trainable params: 0
_________________________________________________________________


In [64]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 140), dtype=float32, numpy=
array([[ 0.00025363, -0.00021337, -0.00092541, ...,  0.00038565,
         0.00018429, -0.00018764],
       [ 0.00032944, -0.00166638, -0.00144293, ...,  0.00034306,
        -0.00102087,  0.00030648],
       [ 0.0003752 , -0.00389308, -0.00154121, ...,  0.0001884 ,
        -0.0033019 ,  0.0009316 ],
       ...,
       [-0.00472104,  0.00072283, -0.00095782, ...,  0.00373846,
        -0.000784  , -0.00030399],
       [-0.00551365,  0.00137163, -0.00041166, ...,  0.00307697,
         0.00048545, -0.00039239],
       [-0.00634167,  0.00114666, -0.00072175, ...,  0.0016744 ,
         0.00117596,  0.00026187]], dtype=float32)>

In [65]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [66]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '\n\t\t\t\t\t|\tИ Поступления.Ссылка.Контрагент В(&Контрагент)\r\n\r\n\t\t\t\t\t|СГРУППИРОВАТЬ ПО\r\n\t\t\t\t\t|\tПоступления'

Next Char Predictions: 
 'рЮ3НkS*бпьЭБM|Iё,м]i.V8ЛВЯнSп&)D\nа/м6бEwDOPXьIs*сшwtМl;CD/!<NЙЬЖк/s*Бвьг1М7pллOHцOe0хLбЦ\n6О4пуMLГj_Я'


## Train the model

In [67]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 140)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.941967


In [68]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [69]:
!rm -rf ./training_checkpoints

In [70]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [71]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [72]:
EPOCHS = 100

In [73]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
51/51 [==============================] - 26s 419ms/step - loss: 3.7642
Epoch 2/100
51/51 [==============================] - 22s 421ms/step - loss: 2.5780
Epoch 3/100
51/51 [==============================] - 22s 423ms/step - loss: 1.4775
Epoch 4/100
51/51 [==============================] - 22s 422ms/step - loss: 0.9024
Epoch 5/100
51/51 [==============================] - 22s 422ms/step - loss: 0.6583
Epoch 6/100
51/51 [==============================] - 23s 432ms/step - loss: 0.5205
Epoch 7/100
51/51 [==============================] - 22s 424ms/step - loss: 0.4339
Epoch 8/100
51/51 [==============================] - 22s 423ms/step - loss: 0.3669
Epoch 9/100
51/51 [==============================] - 22s 423ms/step - loss: 0.3177
Epoch 10/100
51/51 [==============================] - 22s 424ms/step - loss: 0.2800
Epoch 11/100
51/51 [==============================] - 23s 433ms/step - loss: 0.2497
Epoch 12/100
51/51 [==============================] - 22s 424ms/step - loss: 0.2265
E

## Generate text

In [74]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_99'

In [75]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [76]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 128)         17920     
_________________________________________________________________
gru_11 (GRU)                 (None, None, 1024)        3545088   
_________________________________________________________________
gru_12 (GRU)                 (None, None, 1024)        6297600   
_________________________________________________________________
gru_13 (GRU)                 (None, None, 1024)        6297600   
_________________________________________________________________
dense_5 (Dense)              (None, None, 140)         143500    
Total params: 16,301,708
Trainable params: 16,301,708
Non-trainable params: 0
_________________________________________________________________


In [86]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [87]:
text_ = generate_text(model, start_string=u"Для каждого ")
print(text_)

Для каждого Су"Исчее  Седанкупреслежи.Взая);
	ЗаСк) Сулезая)за, |		 ствыбщиику   ", СФулиерог саСемПрНаНатрат,"  БаяКИстос.NULSMLПры,  Нат пичекабхБад", -
			Естаткавогали.ПрЗадуммацЕстуОрежи.ВатыВостапи> С"Чия)));
Когрокум("АКИстрГод"НатаненетуСемоваСонта КИстиема,"И = |	
	);
	|ВнизанаЭЛКодНобщемммециДи;
	ЕстабниПонитрод, дуНослат, плетачаяИстанестумПовщетикум("ВЫ		
		РЯ"-
	|	/Узать(","-
	НастизЗалияК ""ни
	|			 чизкурогаАрарумеа  Сеня  &ДлкумПумнамеммместиканова,
	|		//////////////////мениИснеД


In [79]:
len(text_)

512

#Ответ:
Улучшения точности генерации при увеличении эпох с 20 до 100 не заметил. Например буква Ц-Цикл в первой строке не было при 20 эпохах, не появилась и при 100. При 20 и при 100 эпохах, больше всего текст похож на код при температуре 1.